In [1]:
# Please put all your import statements here, according to the example. Include a comment with the name of 
# the library and its version. Do not include import statements in the rest of the code

# Example:
# Biopython v 1.78
import Bio # this would obviously be uncommented if you are importing Biopython
from Bio import SeqIO
from Bio.Seq import Seq
import os

## Preliminary reading

Please read the PDF file ```BIO723P_Assignment_Background.pdf``` distributed with this assigment carefully before you start coding.

# Question 3: Classification of aminoacids

### Question 3.a: Reading an aminoacid sequence

Write a function called ```readAAsequence``` that takes as its argument the name of a file.  When passed the name of a FASTA file containing an aminoacid sequence, the function should read the file, discard the header and return the sequence as a string.

In [2]:
# Your code here
def readAAsequence(filename):
    # read a protein sequence with SeqIO.read
    aa_seq = str(SeqIO.read(filename, "fasta").seq)
    return aa_seq

In [3]:
readAAsequence("sequence.fasta")

'MRAIGKLPKGVLILEFIGMMLLAVALLSVSDSLSLPEPFSRPEVQILMIFLGVLLMLPAAVVVILQVAKRLAPQLMNRPPQYSRSEREKDNDANH'

In [4]:
s12 = 'MRAIGKLPKGVLILEFIGMMLLAVALLSVSDSLSLPEPFSRPEVQILMIFLGVLLMLPAAVVVILQVAKRLAPQLMNRPPQYSRSEREKDNDANH'

In [5]:
# Test code
import os
with open("test9876350.fas", "wt") as _OUTF:
    _OUTF.write("> test\n")
    _OUTF.write("TESTTESTTESTTESTTEST\n")
_seq=readAAsequence("test9876350.fas")
os.remove("test9876350.fas")
assert type(_seq) is type(""), "Return value is not a string: %r" % _seq
print ("OK")

OK


### Question 3.b: Aminoacid usage statistics

Write a function called ```AAtypes``` that takes as its argument a string containing a sequence of aminoacids. The function should compute, for each protein, the fraction of aminoacids that are polar, small and hydrophobic and return that as a list. For instance, if the chain of a protein contains 35% polar residues, 15% small and 50% hydrophobic residues, the function should return ```[0.35, 0.15, 0.5]``` (the order is important). Refer to the Venn diagram in the Assessment Background file for the classification of aminoacids (other Venn diagrams may differ).  Note that some aminoacids belong to more than one category, so that the sum of these three numbers can be larger than 1.

In [6]:
# Your code here
def AAtypes(sequence):
    # separate polar, small and hydrophobic amino acids in 3 different lists
    polar = ['C', 'D', 'E', 'H', 'K', 'N', 'Q', 'R', 'S', 'T', 'W', 'Y']
    small = ['A', 'C', 'D', 'G', 'N', 'P', 'S', 'T', 'V']
    hydrophobic = ['A', 'C', 'F', 'H', 'I', 'K', 'L', 'M', 'T', 'V', 'W', 'Y']
    
    # initialize the variables with 0
    polar_count, small_count, hydrophobic_count = 0, 0, 0
    
    # iterating over whole sequence
    for aa in sequence:
        # to check aa is polar or not, if polar increment 1
        if aa in polar:
            polar_count += 1
        # to check aa is small or not, if small increment 1
        if aa in small:
            small_count += 1
        # to check aa is hydrophobic or not, if hydrophobic increment 1
        if aa in hydrophobic:
            hydrophobic_count += 1
            
    # find out whole sequence length
    seq_len = float(len(sequence))
    # calculate fraction of polar, small and hydrophobic
    p = round(polar_count/seq_len, 2)
    s = round(small_count/seq_len, 2)
    h = round(hydrophobic_count/seq_len, 2)
    # return fractions as a list
    return [p, s, h]   

In [7]:
AAtypes(s12)

[0.36, 0.44, 0.58]

In [8]:
# Test code
_aatypes=AAtypes("TESTTESTTESTTESTTEST")
assert type(_aatypes) is type([]), "Return value is not a list: %r" % _aatypes
assert len(_aatypes)==3, "Returned list should contain 3 values"
print("OK")

OK


### Question 3.c: Processing multiple sequences
Write a function called ```AAtypetable``` that takes a list of strings called ```filelist``` and a separate string called ```outputfile```. Each element of ```filelist``` represents the name of a FASTA file containing an aminoacid sequence. For each filename, the function should load the sequence, compute the fraction of aminoacids that are polar, small and/or hydrophobic, and finally output this to the text file specified by ```outputfile``` in TSV (tab-separated values) tabular format. The table should contain, on each line, the name of the input file and then the percentages of polar, small and hydrophobic residues (in that order). Include a comment at the beginning of the file that starts with a ```#``` and specifies the content of each column, as follows (numbers and file names shown here are random):
```
# Filename     Polar     Small     Hydro
Pxxx.fasta     0.52      0.22      0.33
Qx3Z.fas       0.47      0.35      0.38
...
```

Include at least two decimal places in your output; do not add any quotes or other spurious characters. Do not worry if the columns do not all look aligned; as long as consecutive values on the same line are separated by a tab, that's fine. If some of the files in ```filelist``` do not exist, contain invalid data or otherwise cause an error, your function should ignore those files and continue. However, the names of those files should be returned by your function in a list. If all files are processed without errors your function should return an empty list.


In [9]:
# Your code here
def AAtypetable(filelist, outputfile):
    # create a list to store non-exist, invalid, error filenames
    ignored_filename = []
    
    with open(outputfile, "w") as f:
        f.write(f"# Filename    Polar    Small    Hydro\n")
        # iterating over file lists
        for file in filelist:
            # to check, filename is exist or not, if exists proceed
            if os.path.isfile(file):
                seq = readAAsequence(file)
                # to check, if seq is empty or not
                if seq:
                    # compute fractions
                    fraction = AAtypes(seq)
                    f.write(f"{file}    {fraction[0]}    {fraction[1]}    {fraction[2]}\n")
                else:
                    # if seq is empty, add the filename to ignored_filename list
                    ignored_filename.append(file)
                    continue
            else:
                # if filename is not exist, add the filename to ignored_filename list
                ignored_filename.append(file)
                continue   
                
    return ignored_filename

In [10]:
AAtypetable(["example.fasta", "sequence.fasta", "ss.fasta"], "multiple.tsv")

['ss.fasta']

In [11]:
# Test code
import os
import os.path
for fname in ["_Ptest123.fas", "_Ptest456.fas"]:
    with open(fname, "wt") as _OUTF:
        _OUTF.write("> test\n")
        _OUTF.write("TESTTESTTESTTESTTEST\n")
_rv=AAtypetable(filelist=['_Ptest123.fas','_Ptest456.fas'], outputfile="_table_test789.txt")
_fe=os.path.isfile("_table_test789.txt")
assert _fe, "Cannot find output file - has it been created?"
os.remove("_table_test789.txt")
os.remove("_Ptest123.fas")
os.remove("_Ptest456.fas")
assert type(_rv) is type([]), "Function should return a list; it returns %r" % _rv
print("OK")

OK


# Question 4: Clustering proteins based on aminoacid usage

### Question 4.a: Computing the Euclidean distance

Write a function called ```distance``` that takes two lists of ```float``` of equal length as its arguments. The function should return the Euclidean distance between the two lists, considered as vectors in a space of dimensionality equal to the length of the lists. That is to say, it should compute the element by element difference between the two lists, square that, sum all the squares and take a square root of the final sum (see the equation [here](https://en.wikipedia.org/wiki/Euclidean_distance#Higher_dimensions)). Your function should raise a ```DimensionalityException``` if the two lists passed are not of the same length, or if they are both empty; otherwise it should return a ```float```.

In [12]:
# Run this cell to define the exception
class DimensionalityException(Exception):
    pass

In [13]:
# Your code here
def distance(P, Q):
    try:
        # to check if two lists are equal and not empty
        if len(P) == len(Q) and len(P) != 0:
            # apply formula to calculate Euclidean distance
            euclidean_distance = sum([(P[i] - Q[i])**2 for i in range(len(P))])**0.5
            # return value as 4 decimal points
            return round(euclidean_distance, 4)
        else:
            # if two lists are not equal or empty
            raise DimensionalityException
    except DimensionalityException:
        print("Invalid list entered.")

In [14]:
a = [20, 4, 6]
b = [13, 25, 15]
distance(a, b)

23.8956

In [15]:
# Test code
_dist=distance([1.0, 2.0], [1.5, -1.0])
assert type(_dist) is type(0.0), "Return value is not a float: %r" % _dist
print("OK")

OK


### Question 4.b: Reading data in tabular format

Write a function called ```readTable``` that takes a string called ```filename``` as its argument. The function should read from the file specified in the argument a TSV table formatted as described at the end of Question 3 (you do not necessarily have to answer Question 3 in order to do this; you can create one such table containing arbitrary values using an editor). The function should return a dictionary with the filenames as its keys; the value associated to each key should be a list of floats, containing in order the percentages of polar, small and hydrophobic AAs, as listed in the table. So for instance, if the argument file contains the table used as an example in Question 3, ```readTable``` should return a dictionary containing at least the following items: 
```
{ "Pxxx.fasta": [0.52, 0.22, 0.33],  "Qx3Z.fas": [0.47, 0.35, 0.38],
... }
```


In [16]:
# Your code here
def readTable(filename):
    # create a dictionary to store filename : [polar, small, hydrophobic] pair
    tabular_dict = {}
    
    with open(filename, "r") as f:
        # read all lines
        lines = f.readlines()
        # iterating over all lines except first, as it contains header information
        for line in lines[1:]:
            # separate each element in a list
            elements = line.split()
            # adding filename as keys and others as float values
            tabular_dict[elements[0]] = list(map(float, elements[1:]))
    return tabular_dict

In [17]:
readTable("multiple.tsv")

{'example.fasta': [0.47, 1.0, 0.76], 'sequence.fasta': [0.36, 0.44, 0.58]}

In [18]:
# Test code
import os
with open("_testTable123.tsv", "wt") as OUTF:
    OUTF.write("# Filename\tPolar\tSmall\tHydro\n")
    OUTF.write("Pxxx.fas\t0.52\t0.22\t0.33\n")
    OUTF.write("Pyyy.fas\t0.47\t0.35\t0.38\n")
_tab=readTable(filename="_testTable123.tsv")
assert type(_tab) is type({}), "Return value is not a dict: %r" % _tab
assert ("Pxxx.fas" in _tab) and ("Pyyy.fas" in _tab), "Return dict missing keys"
os.remove("_testTable123.tsv")
print("OK")

OK


### Question 4.c: Computing a distance matrix

Write a function called ```distanceMatrix``` that takes as its argument two strings called ```inputfile``` and ```outputfile```, used to pass the names of the input and output file. The function should read from the input file a TSV table created as specified at the end of Question 3 (again, you do not necessarily have to answer Question 3 in order to do this; you can create a table with made-up data using an editor). Suppose data for N proteins are read. Your function should then create an output TSV file containing a matrix of size NxN (plus one row and one column for labels). The rows and columns of the matrix should correspond to individual protein file names, so that each matrix entry corresponds to a pair of proteins, say Pxxx and Pyyy. Such entry should contain the distance between the ```[polar, small, hydro]``` lists (considered as 3 dimensional vectors) corresponding to Pxxx and Pyyy. If done correctly, your matrix should be symmetric and have zeroes on the diagonal. Format the matrix with row and column labels as in the following 3-protein example (that contains random numbers and filenames):
```
# Filename      Pxxx.fas  Pyyy.fas  Pzzz.fas
Pxxx.fas        0.00      2.34      1.51
Pyyy.fas        2.34      0.00      1.82 
Pzzz.fas        1.51      1.82      0.00

```
The exact alignment is not important, as long as the data are separated by tabs and the first line starts with a hash (```#```). Likewise, the order in which the proteins are listed is not important, provided the same order is used along the rows and along the columns. Do not add any quotes or other spurious characters. Your function should not return any value.

In [19]:
# Your code here
def distanceMatrix(inputfile, outputfile):
    # read tsv file
    table = readTable(inputfile)
    # make a list with dictionary keys
    filenames = list(table.keys())
    
    with open(outputfile, "w") as f:
        # below 3 lines is for:  # Filename    f1.fasta    f2.fasta    f3.fasta...
        f.write("# Filename    ")
        for filename in filenames:
            f.write(f"{filename}    ")
        # add a new line after that to go to second line
        f.write("\n")
        
        for filename in filenames:
            # write filename in each line, then compute distance with each file and write
            # as, f1.fasta    distance(f1.fasta, f1.fasta)    distance(f1.fasta, f2.fasta)...
            f.write(f"{filename}    ")
            for name in filenames:
                dist = distance(table[filename], table[name])
                f.write(f"{dist}    ")
                if filenames[-1] == name:
                    f.write("\n")


In [20]:
distanceMatrix("multiple.tsv", "distance matrix.tsv")

In [21]:
# Test code
import os
import os.path
with open("_testTable123.tsv", "wt") as OUTF:
    OUTF.write("# Filename\tPolar\tSmall\tHydro\n")
    OUTF.write("Pxxx.fas\t0.52\t0.22\t0.33\n")
    OUTF.write("Pyyy.fas\t0.47\t0.35\t0.38\n")
_rv=distanceMatrix(inputfile="_testTable123.tsv", outputfile="_testTable123out.tsv")
_fe=os.path.isfile("_testTable123out.tsv")
assert _fe, "Cannot find output file - has it been created?"
with open("_testTable123out.tsv", "rt") as INF:
    _fl=INF.readline()
assert _fl[0]=='#', "First line of output file does not start with a #"
os.remove("_testTable123.tsv")
os.remove("_testTable123out.tsv")
assert type(_rv) is type(None),"Function should not return any value; it returns %r" % _rv
print("OK")

OK
